### Imports

In [2]:
import json
from tqdm import tqdm
import pandas as pd
import multiprocessing
from functools import partial
import glob


### Read JSON data

In [12]:
with open('../data/checkin.json', encoding='utf-8') as data_file:
    checkin_data = data_file.read()

### Parse JSON data and create checkin info for individual restaurants

In [13]:

checkin_list = checkin_data.split('\n')[:-1]

relevant_businesses = pd.read_csv('../data/restaurants_in_Toronto.csv')

def create_df(business):
    if relevant_businesses['business_id'].str.contains(business[16:38]).any():
        checkin_business = json.loads(business)
        tmp_df = pd.DataFrame(checkin_business['date'].split(','), columns = ['datetime'])
        tmp_df['business_id'] = checkin_business['business_id']
        tmp_df.to_csv('../data/checkin/'+checkin_business['business_id']+'.csv', index = False)

pool = multiprocessing.Pool(processes=100)
pool.map(create_df, checkin_list)
pool.close()
pool.join()   
print('done')


done


### Concat checkin info for all restaurants

In [14]:

path = r'../data/checkin' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv('../data/checkin_data_raw.csv', index = False)

### Read all checkin data

In [15]:
raw_data = pd.read_csv('../data/checkin_data_raw.csv')

In [16]:

raw_data['datetime'] = pd.to_datetime(raw_data['datetime'])

In [30]:
raw_data['year'] = raw_data['datetime'].dt.year
raw_data['date'] = raw_data['datetime'].dt.day
raw_data['month'] = raw_data['datetime'].dt.month
raw_data['day_of_week'] = raw_data['datetime'].dt.day_name()
raw_data['hour'] = raw_data['datetime'].dt.hour

In [36]:
raw_data_grouped = raw_data.groupby(['business_id','day_of_week','hour']).size().reset_index(name='checkins')
raw_data_grouped.to_csv('../data/restaurant_Toronto_checkin.csv', index = False)